# Building machine learning models

Let's build our first basic models for mortality prediction & predicting the hospital length of stay! We previously explored the rich content of the MIMIC database, extracted and preprocessed data so that we could use them in a classifier. Now, we'll build a model for these prediction tasks and evaluate them.

For mortality prediction, which is a binary classification problem, we'll experiment with logistic regression, support, decision trees and random forests.

Hospital length of stay is a regression problem, hence we will use least squares regression and linear ridge regression from the Shogun toolbox.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline

In [2]:
# SQL database config
sqluser = ''
dbname = 'MIMIC3'
schema_name = 'mimiciii'
hostname = ''
port = 5432
pwd = getpass.getpass()

········


In [3]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=5432, password=pwd)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [4]:
query = \
"""
-- Staging table #1: CHARTEVENTS
with ce as
(
  select adm.hadm_id
  , min(case when itemid in (211,220045) then valuenum else null end) as HeartRate_Min
  , max(case when itemid in (211,220045)  then valuenum else null end) as HeartRate_Max
  , min(case when itemid in (456,52,6702,443,220052,220181,225312) then valuenum else null end) as MeanBP_Min
  , max(case when itemid in (456,52,6702,443,220052,220181,225312)  then valuenum else null end) as MeanBP_Max
  , min(case when itemid in (615,618,220210,224690)  then valuenum else null end) as RespRate_Min
  , max(case when itemid in (615,618,220210,224690)  then valuenum else null end) as RespRate_Max
  from admissions adm
  left join chartevents chart
    on adm.hadm_id = chart.hadm_id
    and chart.itemid in
    (
    -- HEART RATE
    211, --"Heart Rate"
    220045, --"Heart Rate"

    -- MEAN BLOOD PRESSURE
    456, --"NBP Mean"
    52, --"Arterial BP Mean"
    6702, --Arterial BP Mean #2
    443, --Manual BP Mean(calc)
    220052, --"Arterial Blood Pressure mean"
    220181, --"Non Invasive Blood Pressure mean"
    225312, --"ART BP mean"

    -- RESPIRATORY RATE
    618,--Respiratory Rate
    615,--Resp Rate (Total)
    220210,--Respiratory Rate
    224690 --Respiratory Rate (Total)
    )
  left join icustays ie  
  -- match the tables on the patient identifier
  on ie.icustay_id = chart.icustay_id
  -- and require that the observation be made after the patient is admitted to the ICU
  and chart.charttime >= ie.intime
  -- and *before* their admission time + 1 day, i.e. the observation must be made on their first day in the ICU
  and chart.charttime <= ie.intime + interval '1 day'    
    group by adm.hadm_id
)

-- Table #2: Clinical data + demographics
, co AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, first_careunit
, icu.los as icu_los
, round((EXTRACT(EPOCH FROM (adm.dischtime-adm.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los
, EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 as age_icu_in
, pat.gender
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order
, hospital_expire_flag
, CASE WHEN pat.dod IS NOT NULL 
       AND pat.dod >= icu.intime - interval '6 hour'
       AND pat.dod <= icu.outtime + interval '6 hour' THEN 1 
       ELSE 0 END AS icu_expire_flag
FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
ON adm.hadm_id = icu.hadm_id    
)

-- Table #3: Services
, serv AS
(
SELECT icu.hadm_id, icu.icustay_id, se.curr_service
, CASE
    WHEN curr_service like '%SURG' then 1
    WHEN curr_service = 'ORTHO' then 1
    ELSE 0 END
  as surgical
, RANK() OVER (PARTITION BY icu.hadm_id ORDER BY se.transfertime DESC) as rank
FROM icustays icu
LEFT JOIN services se
 ON icu.hadm_id = se.hadm_id
AND se.transfertime < icu.intime + interval '12' hour
)
-- Table #4: Exclusions
, excl AS
(
SELECT
  co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.hosp_los
  , co.age_icu_in
  , co.gender
  , co.icustay_id_order
  , serv.curr_service
  , co.first_careunit
  , co.hospital_expire_flag
  , co.icu_expire_flag
  , CASE
        WHEN co.icu_los < 2 then 1
    ELSE 0 END
    AS exclusion_los
  , CASE
        WHEN co.age_icu_in < 16 then 1
    ELSE 0 END
    AS exclusion_age
  , CASE 
        WHEN co.icustay_id_order != 1 THEN 1
    ELSE 0 END 
    AS exclusion_first_stay
  , CASE
        WHEN serv.surgical = 1 THEN 1
    ELSE 0 END
    as exclusion_surgical
FROM co
LEFT JOIN serv
  ON  co.icustay_id = serv.icustay_id
  AND serv.rank = 1
)

SELECT ex.subject_id, ex.hadm_id, ex.icustay_id, ex.age_icu_in, ex.first_careunit, ex.gender, 
    ex.hospital_expire_flag, ex.icu_expire_flag, ex.hosp_los, ex.icu_los

, ce.HeartRate_Min
, ce.HeartRate_Max
, ce.MeanBP_Min
, ce.MeanBP_Max
, ce.RespRate_Min
, ce.RespRate_Max

FROM excl ex
left join ce
  on ex.hadm_id = ce.hadm_id
WHERE exclusion_los = 0
AND exclusion_age = 0
AND exclusion_first_stay = 0;

"""

query_output = pd.read_sql_query(query,con).dropna()
query_output.head()

,subject_id,hadm_id,icustay_id,age_icu_in,first_careunit,gender,hospital_expire_flag,icu_expire_flag,hosp_los,icu_los,heartrate_min,heartrate_max,meanbp_min,meanbp_max,resprate_min,resprate_max
0,58526,100001,275225,35.476455,MICU,F,0,0,6.2076,4.2567,75.0,131.0,75.000000,131.000000,14.0,22.0
1,9895,100006,291788,48.917320,MICU,F,0,0,12.0618,4.9776,86.0,136.0,77.333298,128.667007,9.0,29.0
2,23018,100007,217937,73.823462,SICU,F,0,0,7.2965,4.0998,55.0,111.0,63.000000,129.000000,9.0,27.0
3,533,100009,253656,60.799222,CSRU,M,0,0,4.9035,2.4908,46.0,92.0,50.000000,82.000000,0.0,37.0
4,87977,100011,214619,21.504107,TSICU,M,0,0,14.3979,11.5029,68.0,142.0,-17.000000,150.000000,0.0,45.0


In [ ]:
query_output.to_csv('./basic-icu-patient-features.csv')

## Mortality prediction

We'll first start with mortality prediction i.e. predicting if the patient died in the hospital. This can be found using the hospital_expire_flag column where 1 indicates that the patient died in the hospital or 0 otherwise. We'll first start with a basic training set of 70% of the entire data, and then testing the performance of our model on the remaining subset.

In [5]:
#query_output = pd.read_csv('./basic-icu-patient-features.csv')

query_output.first_careunit = pd.Categorical(query_output.first_careunit)
query_output.gender = pd.Categorical(query_output.gender)
query_output['gender'] = query_output.gender.cat.codes
query_output['first_careunit'] = query_output.first_careunit.cat.codes

features = ['age_icu_in', 'hosp_los', 'icu_los', 'heartrate_min', \
            'heartrate_max', 'meanbp_min', 'meanbp_max', 'resprate_min', 'resprate_max',
            'gender', 'first_careunit']
X = query_output.loc[:, features] 
y = query_output['hospital_expire_flag'].replace(0, -1)

#X.to_csv('./basic-features.csv', index=False, header=False, sep=' ')
#y.to_csv('./basic-labels.csv', index=False)
X.head()

,age_icu_in,hosp_los,icu_los,heartrate_min,heartrate_max,meanbp_min,meanbp_max,resprate_min,resprate_max,gender,first_careunit
0,35.476455,6.2076,4.2567,75.0,131.0,75.000000,131.000000,14.0,22.0,0,2
1,48.917320,12.0618,4.9776,86.0,136.0,77.333298,128.667007,9.0,29.0,0,2
2,73.823462,7.2965,4.0998,55.0,111.0,63.000000,129.000000,9.0,27.0,0,3
3,60.799222,4.9035,2.4908,46.0,92.0,50.000000,82.000000,0.0,37.0,1,1
4,21.504107,14.3979,11.5029,68.0,142.0,-17.000000,150.000000,0.0,45.0,1,4


### Loading & preprocessing

In [47]:
from modshogun import *

split = int(len(X) * 0.7)

X_train = RealFeatures(np.array(X[:split].T))
X_test = RealFeatures(np.array(X[split:].T))

y_train = BinaryLabels(np.array(y[:split]))
y_test = BinaryLabels(np.array(y[split:]))

print("Number of training samples:", y_train.get_num_labels())
print("Number of testing samples:", y_test.get_num_labels())

('Number of training samples:', 14133)
('Number of testing samples:', 6058)


## Training & evaluating our models

### Linear SVM

In [52]:
%%time

# Parameters to svm
C = 0.1
epsilon = 0.001

svm = LibLinear(C, X_train, y_train)
svm.set_liblinear_solver_type(L2R_L2LOSS_SVC)
svm.set_epsilon(epsilon)

svm.train()

y_pred = svm.apply(X_test)

#use AccuracyMeasure to get accuracy
acc = AccuracyMeasure()
acc.evaluate(y_pred, y_test)

accuracy = acc.get_accuracy() * 100
print('Accuracy(%):', accuracy)

roc = ROCEvaluation()
roc.evaluate(y_pred, y_test)

auc = roc.get_auROC()
print("Area under ROC(%):", auc)

('Accuracy(%):', 90.39286893364147)
('Area under ROC(%):', 0.7787750414205826)
CPU times: user 43.1 s, sys: 43.5 s, total: 1min 26s
Wall time: 32.7 s


### Random forest

In [162]:
y_train = MulticlassLabels(np.array(y.replace(-1, 0)[:split]).astype(np.float))
y_test = MulticlassLabels(np.array(y.replace(-1, 0)[split:]).astype(np.float))

m_vote = MajorityVote()
rand_forest = RandomForest(X_train, y_train, 100)
rand_forest.set_combination_rule(m_vote)
rand_forest.train()
y_pred = rand_forest.apply(X_test)

#use AccuracyMeasure to get accuracy
acc = MulticlassAccuracy()
accuracy = acc.evaluate(y_pred, y_test)

print('Accuracy(%):', accuracy)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: [WARN] In file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/multiclass/tree/CARTree.cpp line 274: Feature types are not specified. All features are considered as continuous in training
  import sys


('Accuracy(%):', 0.9189501485638825)


### Logistic regression

In [147]:
y_train = MulticlassLabels(np.array(y.replace(-1, 0)[:split]).astype(np.float))
y_test = MulticlassLabels(np.array(y.replace(-1, 0)[split:]).astype(np.float))

lr_clf = MulticlassLogisticRegression(1, X_train, y_train)
lr_clf.train()

y_pred = lr_clf.apply_multiclass(X_test)

'''
roc = ROCEvaluation()
roc.evaluate(BinaryLabels(y_pred.get_labels()), BinaryLabels(y_test.get_labels()))
auc = roc.get_auROC()
print("Area under ROC(%):", auc)
'''

#use AccuracyMeasure to get accuracy
acc = MulticlassAccuracy()
accuracy = acc.evaluate(y_pred, y_test)

print('Accuracy(%):', accuracy)

('Accuracy(%):', 0.8648068669527897)


## Using Stratified cross-validation
As seen previously, accuracy is not the best performance metric in this case, since our model predicted 90% accurately whether patients died or not in the hospital. Because of the low number of patients who actually died, predicting 0 or no death every time would yield a high accuracy. For that reason, we'll use the area under the [ROC curve](http://en.wikipedia.org/wiki/Receiver_operating_characteristic) which looks at the false positive and false negative rate. 

Additionally, we'll use [stratified cross-validation](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29#Common_types_of_cross-validation) to get a better estimate the performance of our model. More specifically, stratified cross-validation consists of dividing the data in to k equally sized folds in such a way that labels in each partition is roughly the same. We'll aggregate all the results for every fold and compute the mean which will be the score for the model.

In [45]:
%%time

# Parameters to svm
C = 0.1

k = 10
stratified_split = StratifiedCrossValidationSplitting(labels, k)

metric = ROCEvaluation()

features = RealFeatures(np.array(X))
labels = BinaryLabels(np.array(y))

stratified_split.build_subsets()

aucs = []
for i in range(k):
    train_idx = stratified_split.generate_subset_inverse(i)
    test_idx = stratified_split.generate_subset_indices(i)
    X_train = RealFeatures(np.array(X.loc[train_idx]).T)
    y_train = BinaryLabels(np.array(y.loc[train_idx]))    
    
    X_test = RealFeatures(np.array(X.loc[test_idx]).T)
    y_test = BinaryLabels(np.array(y.loc[test_idx]))
    
    svm = LibLinear(C, X_train, y_train)
    svm.set_liblinear_solver_type(L2R_L2LOSS_SVC)
    
    svm.train()
    
    y_pred = svm.apply(X_test)
    
    roc = ROCEvaluation()
    roc.evaluate(y_pred, y_test)

    auc = roc.get_auROC()
    print("Fold", i, " auROC(%): ", auc)    
    aucs.append(auc)
    
print("Mean auROC across all folds:", np.mean(aucs))

('Fold', 0, ' auROC(%): ', 0.7918805533443827)
('Fold', 1, ' auROC(%): ', 0.7550688073394546)
('Fold', 2, ' auROC(%): ', 0.7850660221794231)
('Fold', 3, ' auROC(%): ', 0.7553070668006487)
('Fold', 4, ' auROC(%): ', 0.777768723005459)
('Fold', 5, ' auROC(%): ', 0.766235332734843)
('Fold', 6, ' auROC(%): ', 0.7704204569621927)
('Fold', 7, ' auROC(%): ', 0.8097944073787421)
('Fold', 8, ' auROC(%): ', 0.7716228136227703)
('Fold', 9, ' auROC(%): ', 0.8117444707501201)
('Mean auROC across all folds:', 0.77949086541180368)


In [10]:
features = RealFeatures(np.array(X).T)
labels = BinaryLabels(np.array(y))

k = 10
stratified_split = StratifiedCrossValidationSplitting(labels, k)

C = 0.1
clf = LibLinear(C, features, labels)
clf.train()

metric = ROCEvaluation()

cv = CrossValidation(clf, features, labels, stratified_split, metric)

result = cv.evaluate()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: [WARN] In file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/classifier/svm/LibLinear.cpp line 422: reaching max number of iterations
Using -s 2 may be faster(also see liblinear FAQ)


  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: [WARN] In file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/evaluation/CrossValidation.cpp line 103: LibLinear does not support locking. Autolocking is skipped. Set autolock flag to false to get rid of warning.

  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: [WARN] In file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/classifier/svm/LibLinear.cpp line 422: reaching max number of iterations
Using -s 2 may be faster(also see liblinear FAQ)


  from ipykernel import kernelapp as app


SystemError: [1;31m[ERROR][0m In file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/labels/Labels.cpp line 67: assertion m_current_values.vector && idx < get_num_labels() failed in virtual float64_t shogun::CLabels::get_value(int32_t) file /build/shogun-v9ad6W/shogun-6.0.0+1SNAPSHOT201704270057/src/shogun/labels/Labels.cpp line 67


# Hospital length of stay

In [164]:
#query_output = pd.read_csv('./basic-icu-patient-features.csv')

query_output.first_careunit = pd.Categorical(query_output.first_careunit)
query_output.gender = pd.Categorical(query_output.gender)
query_output['gender'] = query_output.gender.cat.codes
query_output['first_careunit'] = query_output.first_careunit.cat.codes

features = ['age_icu_in', 'icu_los', 'heartrate_min', \
            'heartrate_max', 'meanbp_min', 'meanbp_max', 'resprate_min', 'resprate_max',
            'gender', 'first_careunit']
X = query_output.loc[:, features] 
y = query_output['hosp_los'].replace(0, -1)

#X.to_csv('./basic-features.csv', index=False, header=False, sep=' ')
#y.to_csv('./basic-labels.csv', index=False)
X.head()

,age_icu_in,icu_los,heartrate_min,heartrate_max,meanbp_min,meanbp_max,resprate_min,resprate_max,gender,first_careunit
0,35.476455,4.2567,75.0,131.0,75.000000,131.000000,14.0,22.0,0,2
1,48.917320,4.9776,86.0,136.0,77.333298,128.667007,9.0,29.0,0,2
2,73.823462,4.0998,55.0,111.0,63.000000,129.000000,9.0,27.0,0,3
3,60.799222,2.4908,46.0,92.0,50.000000,82.000000,0.0,37.0,1,1
4,21.504107,11.5029,68.0,142.0,-17.000000,150.000000,0.0,45.0,1,4


In [170]:
split = int(len(X) * 0.7)

X_train = RealFeatures(np.array(X[:split].T))
X_test = RealFeatures(np.array(X[split:].T))

y_train = RegressionLabels(np.array(y[:split]))
y_test = RegressionLabels(np.array(y[split:]))

print("Number of training samples:", y_train.get_num_labels())
print("Number of testing samples:", y_test.get_num_labels())

('Number of training samples:', 14133)
('Number of testing samples:', 6058)


In [179]:
%%time

# Load models
ls = LeastSquaresRegression(X_train, y_train)

tau = 1
rr = LinearRidgeRegression(tau, X_train, y_train)

# Train
ls.train()
rr.train()

metric = MeanSquaredError()

y_pred = ls.apply_regression(X_test)
mse = metric.evaluate(y_pred, y_test)
print("Least square regression MSE:", mse)
y_pred = rr.apply_regression(X_test)
mse = metric.evaluate(y_pred, y_test)
print("Linear ridge regression MSE:", mse)

('Least square regression MSE:', 76.58140609274638)
('Linear ridge regression MSE:', 76.58140886291336)
CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 91.5 ms
